# IMDb Dataset Loader
Script to load IMDb TSV datasets into PostgreSQL database

In [1]:
import os
import psycopg2

In [ ]:
def getenv_required(key: str) -> str:
    v = os.getenv(key)
    if not v:
        raise RuntimeError(f"Missing environment variable: {key}")
    return v

PGHOST     = getenv_required("PGHOST")
PGPORT     = int(getenv_required("PGPORT"))      # int conversion
PGDATABASE = getenv_required("PGDATABASE")
PGUSER     = getenv_required("PGUSER")
PGPASSWORD = getenv_required("PGPASSWORD")

try:
    conn = psycopg2.connect(
        host=PGHOST,
        port=PGPORT,
        dbname=PGDATABASE,
        user=PGUSER,
        password=PGPASSWORD,
    )
    conn.autocommit = True  
except psycopg2.Error as e:
    print(f"Database connection failed: {e}")
    raise

## Table name_basics

### Create Table

In [3]:
def create_name_basics_table(conn):
    with conn.cursor() as cur:
        # Drop existing table
        cur.execute("""
        DROP TABLE IF EXISTS public.name_basics;
        """)
        
        # Create new table
        cur.execute("""
        CREATE TABLE public.name_basics (
            nconst VARCHAR(10) PRIMARY KEY,
            primaryname VARCHAR(255),
            birthyear INTEGER,
            deathyear INTEGER,
            primaryprofession TEXT,
            knownfortitles TEXT
        );
        """)
    
create_name_basics_table(conn)

### Load data

In [4]:
copy_sql = r"""
COPY public.name_basics (nconst, primaryname, birthyear, deathyear, primaryprofession, knownfortitles)
FROM STDIN
WITH (
  FORMAT CSV,           
  DELIMITER E'\t',      
  NULL '\N',           
  HEADER TRUE,           
  QUOTE E'\b'          -- Using backspace as quote character (\b) effectively disables quote parsing 
                       -- since backspace never appears in the data. This prevents errors with 
                       -- unescaped quotes (") in text fields
);
"""

with conn.cursor() as cur, open(r"C:\GIT projects\IMDb\archive\name.basics.tsv", "r", encoding="utf-8", errors="replace") as f:
    cur.copy_expert(copy_sql, f)

## Table title_akas

### Create Table

In [5]:
def create_title_akas_table(conn):
    with conn.cursor() as cur:
        # Drop existing table
        cur.execute("""
        DROP TABLE IF EXISTS public.title_akas;
        """)
        
        # Create new table
        cur.execute("""
        CREATE TABLE public.title_akas (
            titleid VARCHAR(10),
            ordering INTEGER,
            title VARCHAR(1024),
            region VARCHAR(4),
            language VARCHAR(4),
            types VARCHAR(128),
            attributes VARCHAR(1024),
            isoriginaltitle BOOLEAN,
            PRIMARY KEY (titleid, ordering)
        );
        """)

create_title_akas_table(conn)

### Load data

In [6]:
copy_sql = r"""
COPY public.title_akas (titleid, ordering, title, region, language, types, attributes, isoriginaltitle)
FROM STDIN
WITH (
  FORMAT CSV,           
  DELIMITER E'\t',    
  NULL '\N',            
  HEADER TRUE,            
  QUOTE E'\b'            -- See explanation in name_basics COPY command
);
"""

with conn.cursor() as cur, open(r"C:\GIT projects\IMDb\archive\title.akas.tsv", "r", encoding="utf-8", errors="replace") as f:
    cur.copy_expert(copy_sql, f)

## Table title_basics

### Create Table

In [7]:
def create_title_basics_table(conn):
    with conn.cursor() as cur:
        # Drop existing table
        cur.execute("""
        DROP TABLE IF EXISTS public.title_basics;
        """)
        
        # Create new table
        cur.execute("""
        CREATE TABLE public.title_basics (
            tconst VARCHAR(10) PRIMARY KEY,
            titletype VARCHAR(20),
            primarytitle VARCHAR(1024),
            originaltitle VARCHAR(1024),
            isadult BOOLEAN,
            startyear INTEGER,
            endyear INTEGER,
            runtimeminutes INTEGER,
            genres TEXT
        );
        """)

create_title_basics_table(conn)

### Load data

In [8]:
copy_sql = r"""
COPY public.title_basics (tconst, titletype, primarytitle, originaltitle, isadult, 
                         startyear, endyear, runtimeminutes, genres)
FROM STDIN
WITH (
    FORMAT CSV,         
    DELIMITER E'\t',     
    NULL '\N',            
    HEADER TRUE,          
    QUOTE E'\b'           -- See explanation in name_basics COPY command
);
"""

with conn.cursor() as cur, open(r"C:\GIT projects\IMDb\archive\title.basics.tsv", "r", encoding="utf-8", errors="replace") as f:
    cur.copy_expert(copy_sql, f)

## Table title_principals

### Create Table

In [9]:
def create_title_principals_table(conn):
    with conn.cursor() as cur:
        # Drop existing table
        cur.execute("""
        DROP TABLE IF EXISTS public.title_principals;
        """)
        
        # Create new table
        cur.execute("""
        CREATE TABLE public.title_principals (
            tconst VARCHAR(10),
            ordering INTEGER,
            nconst VARCHAR(10),
            category VARCHAR(50),
            job TEXT,
            characters TEXT,
            PRIMARY KEY (tconst, ordering)
        );
        """)

create_title_principals_table(conn)

### Load data

In [10]:
copy_sql = r"""
COPY public.title_principals (tconst, ordering, nconst, category, job, characters)
FROM STDIN
WITH (
    FORMAT CSV,          
    DELIMITER E'\t',    
    NULL '\N',           
    HEADER TRUE,       
    QUOTE E'\b'           -- See explanation in name_basics COPY command
);
"""

with conn.cursor() as cur, open(r"C:\GIT projects\IMDb\archive\title.principals.tsv", "r", encoding="utf-8", errors="replace") as f:
    cur.copy_expert(copy_sql, f)

## Table title_ratings

### Create Table

In [11]:
def create_title_ratings_table(conn):
    with conn.cursor() as cur:
        # Drop existing table
        cur.execute("""
        DROP TABLE IF EXISTS public.title_ratings;
        """)
        
        # Create new table
        cur.execute("""
        CREATE TABLE public.title_ratings (
            tconst VARCHAR(10) PRIMARY KEY,
            averagerating NUMERIC(3,1),
            numvotes INTEGER
        );
        """)

create_title_ratings_table(conn)

### Load data

In [12]:
copy_sql = r"""
COPY public.title_ratings (tconst, averagerating, numvotes)
FROM STDIN
WITH (
    FORMAT CSV,           
    DELIMITER E'\t',     
    NULL '\N',           
    HEADER TRUE,         
    QUOTE E'\b'           -- See explanation in name_basics COPY command
);
"""

with conn.cursor() as cur, open(r"C:\GIT projects\IMDb\archive\title.ratings.tsv", "r", encoding="utf-8", errors="replace") as f:
    cur.copy_expert(copy_sql, f)

In [ ]:
if conn:
    conn.close()
    print("Database connection closed successfully")